# Tests of simqso

This notebook is a sandbox notebook for testing various aspects of simqso.

In [1]:
from desisim.templates import SIMQSO
qq = SIMQSO()

In [2]:
%time flux1, wave1, meta1, qmeta1 = qq.make_templates(5, seed=123, return_qsometa=True)

CPU times: user 3.59 s, sys: 72.4 ms, total: 3.67 s
Wall time: 3.73 s


In [3]:
%time flux2, wave2, meta2, qmeta2 = qq.make_templates(input_meta=meta1, input_qsometa=qmeta1, return_qsometa=True)

CPU times: user 3.21 s, sys: 53.5 ms, total: 3.27 s
Wall time: 3.33 s


In [4]:
meta1

OBJTYPE,SUBTYPE,TEMPLATEID,SEED,REDSHIFT,MAG,FLUX_G,FLUX_R,FLUX_Z,FLUX_W1,FLUX_W2,OIIFLUX,HBETAFLUX,EWOII,EWHBETA,D4000,VDISP,OIIDOUBLET,OIIIHBETA,OIIHBETA,NIIHBETA,SIIHBETA,ZMETAL,AGE,TEFF,LOGG,FEH
,,,,,mag,nanomaggies,nanomaggies,nanomaggies,nanomaggies,nanomaggies,erg / (cm2 s),erg / (cm2 s),Angstrom,Angstrom,,km / s,,dex,dex,dex,dex,,Gyr,K,m / s2,
str10,str10,int64,int64,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32
QSO,LYA,0,2489768049,2.9181,21.0712,2.62601,3.72841,5.15247,13.8301,20.216,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
QSO,LYA,1,2489768049,3.01703,20.2215,7.20326,8.15488,7.63277,7.88002,10.2964,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
QSO,LYA,2,2489768049,2.21892,21.3603,2.72426,2.85675,4.51482,10.2367,16.5935,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
QSO,LYA,3,2489768049,3.2301,21.5748,1.90613,2.34462,2.15278,3.63474,5.16752,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
QSO,LYA,4,2065586814,1.93824,20.6497,5.28026,5.49709,6.91884,14.3462,22.0883,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0


In [5]:
meta2

OBJTYPE,SUBTYPE,TEMPLATEID,SEED,REDSHIFT,MAG,FLUX_G,FLUX_R,FLUX_Z,FLUX_W1,FLUX_W2,OIIFLUX,HBETAFLUX,EWOII,EWHBETA,D4000,VDISP,OIIDOUBLET,OIIIHBETA,OIIHBETA,NIIHBETA,SIIHBETA,ZMETAL,AGE,TEFF,LOGG,FEH
,,,,,mag,nanomaggies,nanomaggies,nanomaggies,nanomaggies,nanomaggies,erg / (cm2 s),erg / (cm2 s),Angstrom,Angstrom,,km / s,,dex,dex,dex,dex,,Gyr,K,m / s2,
str10,str10,int64,int64,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32
QSO,LYA,0,2489768049,2.9181,21.0721,2.80069,3.72524,4.42753,10.2059,16.9257,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
QSO,LYA,1,2489768049,3.01703,20.2216,6.75939,8.15414,8.55779,15.959,24.7406,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
QSO,LYA,2,2489768049,2.21892,21.3603,3.08668,2.85675,3.69636,4.69248,7.53454,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
QSO,LYA,3,2489768049,3.2301,21.5752,1.91517,2.34379,2.19371,4.43147,6.01558,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
QSO,LYA,4,2065586814,1.93824,20.6498,5.17356,5.49647,6.91769,14.3453,22.087,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0


In [6]:
qmeta1

TEMPLATEID,APPMAG,ABSMAG,SLOPES [5],"EMLINES [62,3]"
int64,float32,float32,float32,float32
0,21.0729,-23.9567,-2.0321 .. -1.24335,1034.51 .. 299.575
1,20.2226,-25.0347,-1.40131 .. -0.926745,1034.26 .. 324.277
2,21.3604,-23.157,-1.10377 .. -0.869287,1034.05 .. 283.654
3,21.5772,-23.6956,-1.2122 .. -1.18499,1035.15 .. 303.622
4,20.6498,-23.7076,-1.95988 .. -1.7997,1034.7 .. 308.063


In [7]:
qmeta2

TEMPLATEID,APPMAG,ABSMAG,SLOPES [5],"EMLINES [62,3]"
int64,float32,float32,float32,float32
0,21.0729,-24.0633,-1.52902 .. -0.94984,1034.53 .. 338.432
1,20.2226,-24.9824,-1.71612 .. -0.805615,1033.4 .. 329.419
2,21.3604,-23.3479,-1.53257 .. -1.03665,1033.69 .. 303.859
3,21.5772,-23.6533,-1.43084 .. -0.912449,1033.77 .. 296.21
4,20.6498,-23.7077,-1.95988 .. -1.7997,1034.7 .. 308.063


In [8]:
stop

NameError: name 'stop' is not defined

In [ ]:
from desisim.test.test_templates import TestTemplates

In [ ]:
tt = TestTemplates()
tt.setUp()

In [ ]:
tt.test_input_meta()

In [ ]:
stop

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

from astropy import cosmology
from simqso.sqgrids import *
from simqso import sqbase
from simqso.sqrun import buildSpectraBulk,buildQsoSpectrum,save_spectra,load_spectra
from simqso.sqmodels import BOSS_DR9_PLEpivot,get_BossDr9_model_vars

In [ ]:
def doit(nqso=3, seed=1, input_qsos=None):

    rand = np.random.RandomState(seed)
    wave = sqbase.fixed_R_dispersion(500, 20e4, 1000)

    cosmo = cosmology.core.FlatLambdaCDM(70.0, 0.3)
    kcorr = sqbase.ContinuumKCorr('DECam-r', 1450, effWaveBand='SDSS-r')
    
    if input_qsos is None:
        zin = rand.uniform(2, 4, nqso)
        qsos = generateQlfPoints(BOSS_DR9_PLEpivot(cosmo=cosmo),
                                 (17, 22), (2.0, 4.0),
                                 kcorr=kcorr, zin=zin,
                                 qlfseed=seed, gridseed=seed)
    else:
        M = AbsMagVar(FixedSampler(input_qsos.absMag), kcorr.restBand)
        m = AppMagVar(FixedSampler(input_qsos.appMag), kcorr.obsBand)
        z = RedshiftVar(FixedSampler(input_qsos.z))
        qsos = QsoSimPoints([M, m, z], cosmo=cosmo, units='flux', seed=seed)

    sedVars = get_BossDr9_model_vars(qsos, wave, noforest=False)
    qsos.addVars(sedVars)
    qsos.loadPhotoMap([('DECam', 'DECaLS'), ('WISE', 'AllWISE')])

    _, spectra = buildSpectraBulk(wave, qsos, saveSpectra=True, maxIter=5, verbose=10)

    return spectra, wave, qsos

In [ ]:
spec1, wave, qsos = doit(seed=555)
spec2, _, qsos2 = doit(seed=555, input_qsos=qsos)
np.all(spec1 == spec2)

In [ ]:
qsos.data

In [ ]:
qsos2.data

In [ ]:
plt.plot(wave, spec1[0, :])
plt.plot(wave, spec2[0, :], alpha=0.5)
plt.xlim(3000, 8000)

In [ ]:
plt.plot(wave, spec1[0, :] / spec2[0, :])
plt.xlim(3000, 8000)

In [ ]:
stop

In [ ]:
spec1, wave, qsos = doit(seed=1)
spec2, _, _ = doit(seed=1)
spec3, _, _ = doit(seed=2)
spec4, _, _ = doit(seed=1)

In [ ]:
assert(np.all(spec1 == spec2))

In [ ]:
assert(np.all(spec3 != spec2))

In [ ]:
assert(np.all(spec4 == spec1))